<a href="https://www.linkedin.com/in/amit-maindola-51801423/"><img src = "https://cdn-images-1.medium.com/max/1200/1*u16a0WbJeckSdi6kGD3gVA.jpeg" width = 400> </a>

<h1 align="center"><font size=5 color="DE5538">Segmenting and Clustering Toronto Neighborhoods</font></h1>

## Introduction
In this Notebook I will be exploring, segmenting and clustering the Neighborhoods in Toronto. 
The Neighborhoods data will loaded from the CSV file created in the previous Notebook

### Table of Contents
<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3 color="black">

1. <a href="#item1">Visualize Toronto Neighborhoods</a>

2. <a href="#item2">Analyze a Neighborhood</a>

2. <a href="#item3">Explore all Neighborhoods</a>
   
3. <a href="#item4">Analyze Each Neighborhood</a>    
   
4. <a href="#item5">Cluster Neighborhoods</a>
    
5. <a href="#item6">Examine Clusters</a>

</font>
</div>

### Import all the required libraries and dependencies

In [40]:
import numpy as np              # library to handle data in a vectorized manner
import pandas as pd             # library for data Analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim   # convert an address into latitude and longitude values

# !conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import requests
import json                     # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print('Libraries imported.')

Libraries imported.


<a id=item1><font size=4 color="229C75">Visualize Toronto Neighborhoods</font></a>

Let's create the data frame from the CSV file created in the previous Notebook.

In [30]:
df=pd.read_csv('toronto_neigh_latlng.csv')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


View the counts of Unique Neighborhoods and Borough

In [31]:
print('The DataFrame has {} Neighborhoods and {} Borough'.format(
    len(df['Neighborhood'].unique())
    ,len(df['Borough'].unique()) 
    )
)

The DataFrame has 103 Neighborhoods and 11 Borough


#### Keep only Borough having **Toronto** in it 

In [32]:
toronto_df = df[df.Borough.str.contains('Toronto')]
# Reset Index
toronto_df.reset_index(drop=True,inplace=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [33]:
print('The DataFrame has {} Neighborhoods and {} Borough'.format(
    len(toronto_df['Neighborhood'].unique())
    ,len(toronto_df['Borough'].unique()) 
    )
)

The DataFrame has 38 Neighborhoods and 4 Borough


#### Use Geopy library to get the Latitude and Longitude of Toronto City

In [34]:
address = "Toronto, Ontario"
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude, longitude = location.latitude, location.longitude
print('The geograpical coordinate of address : "{}" are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of address : "Toronto, Ontario" are 43.653963, -79.387207.


### Create a map of Toronto with Neighborhood imposed

In [35]:
map_toronto = folium.Map(location=(latitude,longitude),zoom_start=10)
for i in toronto_df.index.values:
    lat,lng = toronto_df.at[i,'Latitude'],toronto_df.at[i,'Longitude']
    label = '{}, {}'.format(toronto_df.at[i,'Neighborhood'],toronto_df.at[i,'Borough'])
#     print(label)
    folium.CircleMarker(location=(lat,lng)
                        , radius=5
                        , color='blue'
                        , popup=folium.Popup(label,parse_html=True)
                        , parse_html=False
                        , fill=True
                        , fill_color='#3186cc'
                        , fill_opacity=0.7
                       ).add_to(map_toronto)
    
map_toronto

<a id=item2><font size=4 color="229C75">Analyze a Neighborhood</font></a>

#### Define Foursquare Credentials and Version

In [2]:
# @hidden_cell
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Let's explore the first neighborhood in our dataframe.
Let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [41]:
for i in toronto_df.index.values[0:1]:
    radius = 500
    neighborhood_latitude,neighborhood_longitude = toronto_df.loc[i,'Latitude'],toronto_df.loc[i,'Longitude']
    # let's create the foursquare REST API URL.
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
    #     print(url)    
    results = requests.get(url).json()
    results


In [42]:
venues = results['response']['groups'][0]['items']

# Convert the JSON data to DataFrame
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns, keep only the required fields
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues[filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type,axis=1)

# rename the columns to meaningful
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.678798,-79.298045
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Upper Beaches,Neighborhood,43.680563,-79.292869


In [43]:
print("{} venues returned by FourSquare API".format(nearby_venues.shape[0]))

3 venues returned by FourSquare API


<a id=item2><font size=4 color="229C75">Explore Neighborhoods in Toronto City</font></a>

### Let's create a function to repeat same process to all Neighborhoods

In [44]:
def get_nearby_venues(neighborhoods, latitudes, longitudes, radius = 500 ):
    venues_list=[]
    # Loop for each row
    for name,lat,lng in zip(neighborhoods,latitudes,longitudes):
        #print(name)
        # let's create the foursquare API request URL.
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID
            , CLIENT_SECRET
            , VERSION
            , lat
            , lng
            , radius
            , LIMIT)
        #print(url)    
    
        # Get the result of API
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(name,
                        lat,
                        lng,
                        v['venue']['name'],
                        v['venue']['location']['lat'],
                        v['venue']['location']['lng'],
                        v['venue']['categories'][0]['name']) for v in results])
        
    # Create DataFrame for venues,beak the list of list to individual lists
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    return nearby_venues

In [45]:
toronto_venues = get_nearby_venues(toronto_df['Neighborhood'], toronto_df['Latitude'], toronto_df['Longitude'])
print('Created Venues....')

Created Venues....


Let's view the shape

In [46]:
print(toronto_venues.shape)
toronto_venues.head()

(1705, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
4,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop


Let's see how many Venues returned for each Negighborhood

In [47]:
toronto_venues.groupby('Neighborhood').count()[0:5]

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,53,53,53,53,53,53
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business reply mail Processing Centre969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14


Let's view the unique Categories

In [48]:
toronto_venues.groupby('Venue Category').count()
print("{} unique categories have been returned".format(len(toronto_venues['Venue Category'].unique())))

234 unique categories have been returned


<a id=item3><font size=4 color="229C75">Analyze Each Neighborhood</font></a>

In [49]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column from any position
fixed_columns = list(toronto_onehot.columns)
fixed_columns.remove('Neighborhood')
fixed_columns = ['Neighborhood'] + fixed_columns
fixed_columns
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Let's examine the new dataFrame size

In [50]:
toronto_onehot.shape

(1705, 234)

### Let's take the mean of occurrence various categories

In [51]:
toronto_onehot_grp = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_onehot_grp.head(4)

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.010000,0.01,0.0,0.02,0.0,0.000000,0.0,0.0,0.000000,0.020000,0.0,0.030000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.01,0.0,0.0,0.01,0.020000,0.000000,0.0,0.0,0.020000,0.010000,0.0,0.000000,0.060000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.020000,0.010000,0.070000,0.0,0.0,0.0,0.000000,0.000000,0.020000,0.000000,0.030000,0.0,0.000000,0.0,0.0,0.0,0.02,0.01,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.000000,0.0,0.0,0.010000,0.0,0.000000,0.000

Check the new size

In [52]:
toronto_onehot_grp.shape

(38, 234)

#### View the Top 5 venues for first 3 Neighborhood
Define a method to return the top n venues based on the **default number of venues are 5**

In [180]:
def get_top_venues(df_row,num_top_venues=5,print_row=False):
    row = df_row.T.reset_index()
    neighborhood = row.iloc[0,1]
    # Remove the Neighborhood Value and assign them the column names
    row = row[1:]
    row.columns = ['venue','frequency']
    # Convert the frequency to float, rounded to 2 decimal
    row['frequency'] = row['frequency'].astype(float)
    row = row.round({'frequency':2})
    # Sort the values and reset index
    row.sort_values('frequency',ascending=False, inplace=True)
    row.reset_index(drop=True,inplace=True)
    
    row = row.head(num_top_venues)
    # Print the venues if asked
    if print_row:
        print('{} {} {}'.format('='*4, neighborhood, '='*4))
        print(row.head(5) )
        print('\n')
    return row.venue.tolist()

# Test the method
get_top_venues(toronto_onehot_grp.loc[1])

['Coffee Shop', 'Cocktail Bar', 'Bakery', 'Steakhouse', 'Cheese Shop']

View the top Venues for first 3 Neighborhoods

In [181]:
# Loop for each Neighborhood
for i in toronto_onehot_grp.index.values[0:2]:
    top_venues = get_top_venues(toronto_onehot_grp.loc[i],print_row=True)        

==== Adelaide, King, Richmond ====
                 venue  frequency
0          Coffee Shop       0.07
1                 Café       0.06
2      Thai Restaurant       0.04
3           Steakhouse       0.04
4  American Restaurant       0.04


==== Berczy Park ====
          venue  frequency
0   Coffee Shop       0.09
1  Cocktail Bar       0.06
2        Bakery       0.04
3    Steakhouse       0.04
4   Cheese Shop       0.04




#### Create a new DataFrame with Top 10 Venues

In [183]:
# Form the columns list for new DataFrame
columns=['Neighborhood']
for i in range(1,11):
    columns.append('Rank {}'.format(i))

# Create a new DataFrame
neigh_top_venues = pd.DataFrame(columns=columns)
neigh_top_venues['Neighborhood'] = toronto_onehot_grp['Neighborhood']

# Loop for each Neighborhood and add the venues to DataFrame
for i in toronto_onehot_grp.index.values:
    neigh_top_venues.iloc[i,1:] = get_top_venues(toronto_onehot_grp.loc[i],num_top_venues=10)

# View the new DataFrame
neigh_top_venues.head()

,Neighborhood,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Cosmetics Shop,Gym,Bar,Hotel,Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Cheese Shop,Café,Beer Bar,Restaurant,Seafood Restaurant,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Burrito Place,Caribbean Restaurant,Bar,Furniture / Home Store,Climbing Gym,Stadium,Falafel Restaurant
3,Business reply mail Processing Centre969 Eastern,Yoga Studio,Auto Workshop,Comic Shop,Park,Recording Studio,Restaurant,Butcher,Burrito Place,Farmers Market,Brewery
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Plane,Harbor / Marina,Boat or Ferry,Airport Gate,Airport Food Court,Airport,Boutique


<a id=item4><font size=4 color="229C75">Cluster Neighborhoods</font></a>

Import the Libraries

In [211]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


Use k-means to Cluster the Neighborhoods

In [200]:
# set 5 clusters
kclusters=5
neigh_cluster=toronto_onehot_grp.drop('Neighborhood',axis=1)
neigh_cluster.head()

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neigh_cluster)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])

# Check the unique clusters
print(set(kmeans.labels_))

[0 0 0 0 0 0 0 0 0 0]
{0, 1, 2, 3, 4}


Let's create another DataFrame which contains cluster

In [203]:
# Create cluster dataframe
neigh_merged = toronto_df

# Add clustering labels
neigh_merged['ClusterLabel'] = kmeans.labels_
# Add the top 10 venues of the neighborhood
neigh_merged = neigh_merged.join( neigh_top_venues.set_index('Neighborhood'), on='Neighborhood')

# View the values
neigh_merged.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabel,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Accessories Store,Music Venue,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Bubble Tea Shop,Lounge,Furniture / Home Store,Fruit & Vegetable Store,Spa
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Board Shop,Sushi Restaurant,Pet Store,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,American Restaurant,Italian Restaurant,Diner,Brewery,Cheese Shop,Chinese Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Music Venue,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant


Let's Visualize the clusters

In [220]:
map_clusters = folium.Map(location=[latitude,longitude],zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
for lat, lon, poi, cluster in zip(neigh_merged['Latitude'], neigh_merged['Longitude'], neigh_merged['Neighborhood'], neigh_merged['ClusterLabel']):
    # Add Market
    folium.CircleMarker(
        [lat,lon]
        , radius=10
        , popup=folium.Popup("{} - {}".format(poi,cluster), parse_html=True)
        , fill=True
        , color=rainbow[cluster-1]
        , fill_opacity=0.7).add_to(map_clusters)               

# Display the map    
map_clusters

<a id=item5><font size=4 color="229C75">Examine Clusters</font></a>

Let's Examine the Clusters now to find out the categories distinguishing each cluster.

In [ ]:
new_cols = [neigh_merged.columns[2]]
for col in neigh_merged.columns[6:]:
    new_cols.append(col)
new_cols

### View Cluster 0

In [273]:
neigh_merged.loc[neigh_merged['ClusterLabel'] == 0, new_cols]

,Neighborhood,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10
0,The Beaches,Coffee Shop,Pub,Accessories Store,Music Venue,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
1,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Bubble Tea Shop,Lounge,Furniture / Home Store,Fruit & Vegetable Store,Spa
2,"The Beaches West, India Bazaar",Park,Board Shop,Sushi Restaurant,Pet Store,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,Studio District,Café,Coffee Shop,Gastropub,Bakery,American Restaurant,Italian Restaurant,Diner,Brewery,Cheese Shop,Chinese Restaurant
4,Lawrence Park,Bus Line,Park,Swim School,Dim Sum Restaurant,Music Venue,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
5,Davisville North,Pizza Place,Hotel,Burger Joint,Breakfast Spot,Dog Run,Park,Food & Drink Shop,Sandwich Place,Miscellaneous Shop,Middle Eastern Restaurant
6,North Toronto West,Clothing Store,Sporting Goods Shop,Coffee Shop,Gym / Fitness Center,Chinese Restaurant,Spa,Sandwich Place,Salon / Barbershop,Rental Car Location,Park
7,Davisville,Sandwich Place,Pizza Place,Dessert Shop,Italian Restaurant,Restaurant,Café,Coffee Shop,Sushi Restaurant,Seafood Restaurant,Japanese Restaurant
8,"Moore Park, Summerhill East",Playground,Tennis Court,Park,Restaurant,Accessories Store,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mexican Restaurant
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Convenience Store,Pub,Coffee Shop,Sports Bar,Sushi Restaurant,Supermarket,Fried Chicken Joint,American Restaurant,Bagel Shop,Vietnamese Restaurant


### View Cluster 1

In [274]:
neigh_merged.loc[neigh_merged['ClusterLabel'] == 1, new_cols]

,Neighborhood,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10
17,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Sandwich Place,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Ice Cream Shop,Falafel Restaurant
24,"The Annex, North Midtown, Yorkville",Coffee Shop,Sandwich Place,Café,Pizza Place,Flower Shop,Jewish Restaurant,History Museum,Pub,Park,Burger Joint
27,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Plane,Harbor / Marina,Boat or Ferry,Airport Gate,Airport Food Court,Airport,Boutique


### View Cluster 2

In [266]:
neigh_merged.loc[neigh_merged['ClusterLabel'] == 2, new_cols]

,Neighborhood,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10
22,Roselawn,Garden,Ice Cream Shop,Music Venue,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


### View Cluster 3

In [267]:
neigh_merged.loc[neigh_merged['ClusterLabel'] == 3, new_cols]

,Neighborhood,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10
28,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Café,Cocktail Bar,Hotel,Seafood Restaurant,Restaurant,Beer Bar,Farmers Market,Burger Joint,Cheese Shop


### View Cluster 4

In [268]:
neigh_merged.loc[neigh_merged['ClusterLabel'] == 4, new_cols]

,Neighborhood,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10
35,"Parkdale, Roncesvalles",Breakfast Spot,Gift Shop,Piano Bar,Dessert Shop,Movie Theater,Bar,Burger Joint,Eastern European Restaurant,Restaurant,Dog Run


### Observation

We can observe that there are more than 41 places for Cafe in the Toronto Neighborhoods.  
Toronto Neighborhoods have mostly Eatries places like Coffe Shops, Restaurant, Pub and Hotels in Trend.

In [301]:
neigh_merged['Rank 2'].value_counts()

Café                             8
Coffee Shop                      3
Hotel                            3
Bakery                           2
Pub                              2
Airport Terminal                 1
Vegetarian / Vegan Restaurant    1
Breakfast Spot                   1
Auto Workshop                    1
Trail                            1
Park                             1
Board Shop                       1
Grocery Store                    1
Ice Cream Shop                   1
Pizza Place                      1
Restaurant                       1
Playground                       1
Bar                              1
Sporting Goods Shop              1
Bookstore                        1
Clothing Store                   1
Tennis Court                     1
Sandwich Place                   1
Cocktail Bar                     1
Gift Shop                        1
Name: Rank 2, dtype: int64